In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D
from keras.models import Model
from keras import backend as K
from sklearn.utils import resample

n_features = 4
latent_dim = 10
n_runs = 300
max_len = 0



def prepare_training(path, n_runs):
    labels = []
    def closest_4(n, m):
        q = n / m
        n1 = m * q
        if (n * m) > 0:
            n2 = m * (q + 1)
        else:
            n2 = m * (q - 1)
        if abs(n-n1) < abs(n-n2):
            return int(n1)
        return int(n2)
    
    
    def extend_line(run, max_len):
        difference = abs(len(run) - max_len)
        extension = np.array([run[-1]]*difference)
        if difference != 0:
            run = np.vstack([run, extension])
        return run
    
    def get_max_len(sequence_list):
        max_len = 0
        min_len = 1000
        for seq in sequence_list:
            if len(seq) > max_len:
                max_len = len(seq)
            if len(seq) < min_len:
                min_len = len(seq)
        return max_len, min_len
    
    def construct_matrix(sequence_list):
        max_len, min_len = get_max_len(sequence_list)
        print(max_len)
        len = closest_4(max_len,4)
        len = 440
        train_matrix = np.zeros(shape=(n_runs, len, n_features))
        for index, run in enumerate(sequence_list):
            line = extend_line(run, len)
            train_matrix[index] = line
        return train_matrix
        
        
    def stadard_sequences(seqs):
        for i, seq in enumerate(seqs):
            #seqs[i] = MinMaxScaler(feature_range=[0,1]).fit_transform(seq)
            seqs[i] = StandardScaler().fit_transform(seq)
        return seqs       
    
    
    def order_runs_by_len(runs):
        runs.sort(key=len)
        for r in runs:
            labels.append(r['Choice'][0])
        for i,r in enumerate(runs):
            runs[i] = runs[i].drop(columns=['Choice'])
        return runs, labels
    
    def read_sequences():
        run_list_mix = []
        for index in range(n_runs):
            run_csv = pd.read_csv(path+str(index))
            run_csv = run_csv.drop(columns=['Unnamed: 0'])
            run_list_mix.append(run_csv)
        run_list_ordered, labels = order_runs_by_len(run_list_mix)
        stands = stadard_sequences(run_list_ordered)
        padded_matrix = construct_matrix(stands)
        return padded_matrix, labels
    
    return read_sequences()

train_matrix, labels = prepare_training("Mix_sequences_var_length/run^", n_runs=n_runs) 
labels = np.array(labels)


Using TensorFlow backend.


412


In [7]:
import keras
from keras import objectives
from keras.losses import mse
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Conv1D, Conv2DTranspose, Lambda, MaxPooling1D, UpSampling1D, RepeatVector, \
    Dropout
import keras.backend as K
from keras.optimizers import Adam
from tensorflow_core.python.training import optimizer

filters = 40


def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim), stddev=1.)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


def Conv1DTranspose(input_tensor, filters, kernel_size, dil_rate, strides=1, padding='same'):
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), 
                            strides=(strides, 1), padding=padding,
                            activation='relu',
                            dilation_rate=dil_rate)(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x


# Define an input sequence and process it.
encoder_inputs = Input(shape=(train_matrix.shape[1], n_features), name='Encoder_input')

encoder = LSTM(latent_dim, return_state=True)
x = encoder_inputs
dil_rate = 4
for i in range(4):
     x = Conv1D(filters=filters,
                    kernel_size=10,
                activation='relu',
                dilation_rate=1,
                padding='same')(x)
     # x = Dropout(0.5)(x)
     x = MaxPooling1D(pool_size=2)(x)
     if i == 1:
        shape = K.int_shape(x)
# for i in range(4):
#     x = Dense(200, activation='relu')(x)
#     x = Dropout(0.5)(x)
encoder_outputs, state_h, state_c = encoder(x)
encoder_states = [state_h, state_c]

z_mean = Dense(latent_dim, name='z_mean',)(state_c)
z_log_var = Dense(latent_dim, name='z_log_var')(state_c)
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# variational_space = [z_mean, z_log_var, z]
# shape = K.int_shape(x)

# x = Flatten()(x)
# x = Dense(latent_dim)(x)
encoder = Model(encoder_inputs, [z_mean, z_log_var], name='encoder')
#encoder.summary()

# Set up the decoder, using `encoder_states` as initial state.
#decoder_inputs = Input(shape=(None,n_features))

# latent_inputs = Input(shape=(latent_dim,), name='latent_inputs')
# x = Dense(shape[1]*shape[2])(latent_inputs)
# x = Reshape((shape[1], shape[2]))(x)

decoder_inputs = Input(shape=(train_matrix.shape[1], n_features), name='dec_inputs')
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
x_dec, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# latent_inputs = Input(shape=(latent_dim,))
# x = Dense(shape[1]*shape[2])(latent_inputs)
# x = Reshape((shape[1], shape[2]))(x)
for i in range(2):
        x_dec = Conv1DTranspose(input_tensor=x_dec, filters=filters,
                            kernel_size=10, dil_rate=1, padding='same')
        # x_dec = Dropout(0.5)(x_dec)
        #x_dec = UpSampling1D(size=2)(x_dec)
# for i in range(4):
#     x_dec = Dense(200, activation='relu')(x_dec)
decoder_dense = Dense(n_features, name="Decoder_output")
decoder_outputs = decoder_dense(x_dec)
# 
# def inference_net():
#     decoder_state_input_h = Input(shape=(latent_dim,))
#     decoder_state_input_c = Input(shape=(latent_dim,))
#     decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
#     dec_out, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
#     decoder_states = [state_h, state_c]
#     x_dec = dec_out 
#     for i in range(2):
#         x_dec = Conv1DTranspose(input_tensor=x_dec, filters=filters,
#                             kernel_size=20, dil_rate=1, padding='same')
#         
#     dec_out = decoder_dense(x_dec)
#     decoder_model = Model([decoder_inputs] + decoder_states_inputs, dec_out)
#     return decoder_model
    
def vae_loss(x, x_decoded_mean):
        xent_loss = mse(x, x_decoded_mean) 
        #xent_loss = K.sum(K.binary_crossentropy(x_decoded_mean, x), axis=-1)
        kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var))
        loss = xent_loss + 0*kl_loss
        return loss

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss=vae_loss)
#decoder.summary()
model.summary()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder_input (InputLayer)      (None, 440, 4)       0                                            
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 440, 40)      1640        Encoder_input[0][0]              
__________________________________________________________________________________________________
max_pooling1d_5 (MaxPooling1D)  (None, 220, 40)      0           conv1d_5[0][0]                   
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 220, 40)      16040       max_pooling1d_5[0][0]            
__________________________________________________________________________________________________
max_poolin

In [8]:
model.fit([train_matrix, train_matrix ],train_matrix,
          epochs=50)
model.save_weights("Models/Weights/LSTM_seq2seq_300_500.hdf5")


Epoch 1/50


 32/300 [==>...........................] - ETA: 48s - loss: 1.1915

 64/300 [=====>........................] - ETA: 25s - loss: 1.1319

 96/300 [========>.....................] - ETA: 16s - loss: 1.0801

128/300 [===========>..................] - ETA: 12s - loss: 1.0298

...

192/300 [==================>...........] - ETA: 10s - loss: 0.0041

224/300 [=====================>........] - ETA: 6s - loss: 0.0041 

256/300 [========================>.....] - ETA: 3s - loss: 0.0040

288/300 [===========================>..] - ETA: 0s - loss: 0.0040

300/300 [==============================] - 24s 79ms/step - loss: 0.0041


In [1]:

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

decoder_model.summary()

encoded = encoder.predict(train_matrix)


NameError: name 'Model' is not defined

In [74]:

run = decoder_model.predict([train_matrix,encoded[0], encoded[1]])


(200, 420, 4)


In [30]:
model.load_weights("Models/Weights/LSTM_seq2seq_300_500.hdf5")


In [9]:
from sklearn.decomposition import PCA

encodings = encoder.predict(train_matrix)
enc_mean, enc_var, z_enc = encodings[0], encodings[1], encodings[0]
print(enc_mean.shape, enc_var.shape, z_enc.shape)


def return_mask(num, labs):
    arg = np.squeeze(np.argwhere(labs == num))
    return arg

masks = [return_mask(num, np.array(labels)) for num in range(0, 9)]


from mpl_toolkits.mplot3d import Axes3D  
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

import numpy as np
from scipy.stats import norm


def plot_pca(title, i): 
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    markers = ['o', 'o', 'o', 'o', '^', '^', '^', '^', '^', '^']
    for index, mask in enumerate(masks):
        ax.scatter(principalComponents[:, 0][mask], 
                   principalComponents[:, 1][mask],  
                   principalComponents[:, 2][mask], marker=markers[index])

    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()
    for mask in masks:
        plt.scatter(x=principalComponents[:, 0][mask], 
                    y=principalComponents[:, 1][mask])

        #break
    
    plt.legend(labels=np.arange(0, 9))
    plt.title(str(title))
    plt.show()


enc_list = [enc_mean, enc_var, z_enc]
titles = ["MEAN","LOG_VAR","SAMPLED"]
for i,enc in enumerate(enc_list):
    scaler = StandardScaler()
    enc_input = scaler.fit_transform(enc) 
    pca = PCA(3)
    principalComponents = pca.fit_transform(enc_input)
    print("Component 1 shape", principalComponents.shape)
    print("PCA variance ratio", pca.explained_variance_ratio_) 
    plot_pca('Sequences'+titles[i], 0)


(300, 10) (300, 10) (300, 10)
Component 1 shape (300, 3)
PCA variance ratio [0.6392587  0.27275503 0.07151075]


Component 1 shape (300, 3)
PCA variance ratio [0.78477883 0.16442758 0.04046951]


Component 1 shape (300, 3)
PCA variance ratio [0.6392587  0.27275503 0.07151075]


In [12]:
# 
# unseen_labs = pd.read_csv("Data/Boat_nominal_data/Boat_unseen_labels_mix.csv")
# unseen_labs = unseen_labs.drop(columns="Unnamed: 0") 
# unseen_labs = np.array(unseen_labs)

unseen_sequences_matrix, unseen_labs = prepare_training("Mix_sequences_var_length/run_unseen^", 
                                           n_runs=500)
unseen_mask = [return_mask(num, np.array(unseen_labs)) for num in range(0, 9)]

print(unseen_sequences_matrix.shape)

unseen_encoding = encoder.predict(unseen_sequences_matrix)
print(len(unseen_encoding), unseen_encoding[0].shape)


418
(500, 440, 4)


3 (500, 10)


In [23]:
reconstruction = model.predict([train_matrix, train_matrix])
reconstruction_unseen = model.predict([unseen_sequences_matrix, unseen_sequences_matrix])

#RECONSTRUCTION ERROR
def get_reconstructed_matrix(input_matrix, reconstrut):
    return_matrix = np.zeros(shape=input_matrix.shape)
    for i,run in enumerate(input_matrix):
        mask_seq = np.squeeze(np.argwhere(np.mean(run, axis=1) != 0))
        return_matrix[i][mask_seq] = reconstrut[i][mask_seq]
    return reconstrut


train_error = abs(train_matrix - get_reconstructed_matrix(train_matrix, reconstruction))
unseen_runs_error = abs(unseen_sequences_matrix - get_reconstructed_matrix(train_matrix, reconstruction_unseen))

train_error_avg = np.mean(train_error, axis=2)
unseen_error_avg = np.mean(unseen_runs_error, axis=2)
train_error_avg = np.mean(train_error_avg, axis=1)
unseen_error_avg = np.mean(unseen_error_avg, axis=1)
print(train_error_avg.shape, unseen_error_avg.shape)

for mask in masks:
    plt.scatter(np.linspace(1,n_runs,n_runs)[mask],train_error_avg[mask])
plt.title('ERROR ON TRAIN')
plt.show()

for mask in unseen_mask:
    plt.scatter(np.linspace(1,n_runs,n_runs)[mask],unseen_error_avg[mask])
plt.title('ERROR ON Unseen')
plt.show()

NameError: name 'unseen_sequences_matrix' is not defined

In [ ]:

#RECONSTRUCTION
def reconstruct_sequence(seq_index):
    run = train_matrix[seq_index]
    #mask_seq = np.squeeze(np.argwhere(np.mean(run, axis=1) != 0))
    rec_run = reconstruction[seq_index]#[mask_seq]
    #print(rec_run[:,0])
    df = pd.DataFrame(rec_run[:-15], columns=["Sin", "Cosin", "Lat", "Lon"])
    df_original = pd.DataFrame(run[:-15], columns=["Sin", "Cosin", "Lat", "Lon"])
    plt.plot(df_original['Lon'], df_original['Lat'])
    plt.plot(df['Lon'], df['Lat'])
    plt.show()

for i in range(len(train_matrix)):
    reconstruct_sequence(i)
    if i == 10:
        break


5


In [36]:
from sklearn.metrics.pairwise import cosine_similarity


def calculate_centroids():
    centroids = []
    for index, mask in enumerate(masks):
        centroids.append(encodings[0][mask])
        centroids[index] = np.mean(centroids[index], axis=0)
    return centroids


centroids = calculate_centroids()



5


In [44]:
def strange_new_data():
    strange_data = pd.read_csv("Mix_sequences_var_length/run_strange^0")
    strange_data = strange_data.drop(columns=['Unnamed: 0', 'Choice'])
    st_data = StandardScaler().fit_transform(strange_data)
    tr_matrix = np.zeros((1, train_matrix.shape[1], train_matrix.shape[2]))
    difference = abs(len(st_data) - 440)
    extension = np.array([st_data[-1]]*difference)
    if difference != 0:
        st_data = np.vstack([st_data, extension])
    tr_matrix[0] = st_data
    #print(tr_matrix)
    return tr_matrix
    
        
unseen_strange_matrix = strange_new_data()
enc_strange_mean, enc_strange_var = encoder.predict(unseen_strange_matrix)
# enc_strange_mean = np.squeeze(enc_strange_mean)
# enc_strange_var = np.squeeze(enc_strange_var)

def calculate_class():
    similarities = []
    for cl, cent in enumerate(centroids):
        cent = np.reshape(cent, (-1,10))
        similarities.append(cosine_similarity(enc_strange_mean, cent))
    similarities = np.array(similarities)
    class_anom = np.argmax(similarities)
    return class_anom
    
    
print(calculate_class())


5


In [ ]:
def decode(sampling):

    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)


def interpolate():
    encoder.predict()
    
    
    
    
    
    
    
    